<a href="https://colab.research.google.com/github/jtao22/PythonAI/blob/main/StockMarket/closepriceprediction2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import 
from sklearn.svm import SVR
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot 
from sklearn.preprocessing import MinMaxScaler as mms
import datetime as dt
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression as lr
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.ensemble import RandomForestClassifier as rfc
pyplot.style.use('seaborn-darkgrid')

In [ ]:
#upload data
from google.colab import files
files.upload()


In [6]:
data = pd.read_csv('tesla.csv')
data.head(5)

,Date,Open,High,Low,Close,Volume,Adj Close
0,6/29/2010,19.000000,25.00,17.540001,23.889999,18766300,23.889999
1,6/30/2010,25.790001,30.42,23.299999,23.830000,17187100,23.830000
2,7/1/2010,25.000000,25.92,20.270000,21.959999,8218800,21.959999
3,7/2/2010,23.000000,23.10,18.709999,19.200001,5139800,19.200001
4,7/6/2010,20.000000,20.00,15.830000,16.110001,6866900,16.110001


In [7]:
#filter
Y = data['Adj Close'].values
data = data.drop(labels=['Adj Close', 'Close'], axis = 1)
for i in range(len(data['Date'])):
 temp = data['Date'][i]
 data['Date'][i] = dt.datetime.strptime(temp,"%m/%d/%Y")
data['day'] = data['Date'].apply(lambda r:r.day)
data['month'] = data['Date'].apply(lambda r:r.month)
data['year'] = data['Date'].apply(lambda r:r.year)
data = data.drop(labels=['Date'], axis=1)
scaler = mms(feature_range=(0,1))
data.iloc[:,-7:] = scaler.fit_transform(data.iloc[:,-7:])
data = data.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
#split
trainX,testX,trainY,testY = tts(data, Y, test_size = 0.25, random_state = 42)


In [9]:
#Create SVR
lsvr = SVR(kernel='linear', C=1000.0)
lsvr.fit(trainX,trainY)
psvr = SVR(kernel='poly', C=1000.0, degree=2)
psvr.fit(trainX, trainY)
rsvr = SVR(kernel='rbf', C=1000.0, gamma=0.15)
rsvr.fit(trainX, trainY)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.15,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [14]:
#test
lpredict = lsvr.predict(testX)
ppredict = psvr.predict(testX)
rpredict = rsvr.predict(testX)
lcount = 0
pcount = 0
rcount = 0
total = len(testY)
for i in range(len(testY)):
  if (lpredict[i] <= testY[i]+2 and lpredict[i] >= testY[i]-2):
    lcount = lcount + 1
  if (ppredict[i] <= testY[i]+2 and ppredict[i] >= testY[i]-2):
    pcount = pcount + 1
  if (rpredict[i] <= testY[i]+2 and rpredict[i] >= testY[i]-2):
    rcount = rcount + 1
print("linear:", lcount/total*100,'%')
print("polynomial:", pcount/total*100,'%')
print("rbf:", rcount/total*100,'%')

linear: 82.26950354609929 %
polynomial: 26.95035460992908 %
rbf: 79.66903073286052 %
